**1. Import Libraries**

In [ ]:
import json
import os

**2. File Function**

This section include useful functions for load/unload and check if a file exist

In [ ]:
def _check_file_exist(file_name):
  """
  Verify if file_name exist in current directory
  if not create file_name

  file_name [str] inventory.json or cash_register.json
  """

  #Get current working directory
  current_directory=os.getcwd()

  #List all files and folders in the current directory
  entries=os.listdir(current_directory)

  #verify file_name in current directory
  #if file not exist, create it
  if not file_name in entries:
    if file_name == "inventory.json":
      file_dict={"prodotto":{}}
    elif file_name == "cash_register.json":
      file_dict={"Profitto":{"lordo":0,"netto":0}}
    _update_file(file_dict,file_name)


def _update_file(file_dict,file_name_str):
  """
  Update persistent file

  file_dict[dict]
  file_name_str[str] inventory.json or cash_register.json
  """

  with open(file_name_str, "w") as file_name:
    json.dump(file_dict, file_name, indent=1)


def _load_file(file_name_str):
  """
  Load data

  file_name_str[str] inventory.json or cash_register.json
  """
  #verify file exists, if not create it
  _check_file_exist(file_name_str)

  with open(file_name_str, "r") as file_name:
    file_dict=json.load(file_name)
  return file_dict

**3. String modify function**

This section include functions to modify string based on max lenght of keys/values (from inventory)

In [ ]:
def _find_max_string_len(inventory):
  """
  Find max length of string in dictionary for:
  • product
  • quantity
  • sell price

  inventory [dict] inventory dictionary

  If lenght of all product name in the inventory is < len("Prodotto") the function return len("Prodotto")
  same logic for quantity and sell price
  """
  #list all product name
  product_list=inventory["prodotto"].keys()
  #find max stirng len in product list
  max_len_product=len(max(product_list, key=len))
  #compare max len found with the lenght of string "Prodotto"
  if max_len_product<len("Prodotto"):
    max_len_product=len("Prodotto")

  #set max_len_quantity to length of string "Quantità"
  max_len_quantity=len("Quantità")
  #for every product compare lenght of quantity values and update max lenght quantity
  for prod in inventory["prodotto"].keys():
    if len(str(inventory["prodotto"][prod]["quantità"]))>max_len_quantity:
      max_len_quantity=len(str(inventory["prodotto"][prod]["quantità"]))

  #set max_len_price to length of string "Prezzo"
  max_len_price=len("Prezzo")
  #for every product compare lenght of sell price values and update max lenght price
  for prod in inventory["prodotto"].keys():
    if len(str(inventory["prodotto"][prod]["prezzo di vendita"]))>max_len_price:
      max_len_price=len(str(inventory["prodotto"][prod]["prezzo di vendita"]))

  return max_len_product, max_len_quantity, max_len_price


def _adjust_string(string, max_len):
  """
  Adjust string lenght to max_len adding space at the end

  string [str] string to adjust
  max_len [int] max lenght of string
  """
  string=string+" "*(max_len-len(string))
  return string

**4. Input verify function**

This section includes function used to verify that user input satisfy the requirements

In [ ]:
def _is_valid_input(string,par):
  """
  Insert a valid input. Valid is an input that can be converted in 'par'
  The function will continue ask for a valid input until is provided

  string [str] string to print
  par[str] int or float
  """
  is_ok=False
  while not is_ok:
    try:
      if par=="int":
        input_val=int(input(string))
        if input_val<=0:
          print("Input non valido! Inserire un intero >0")
        else:
          is_ok=True
      elif par=="float":
        input_val=float(input(string))
        if input_val<0:
          print("Input non valido! Inserire un numero >=0")
        else:
          is_ok=True
    except ValueError:
      if par=="int":
        print("Input non valido! Inserire un intero >0")
      elif par=="float":
        print("Input non valido! Inserire un numero >=0")
  return input_val


def _input_available_qauntity(sell_np,inventory):
  """
  Insert available quantity of sell_np
  The function will continue ask for a valid input until is provided or
  until the user decide to stop transaction

  The function return the quantity [int].
  In case of user decide to stop transaction, return 0

  sell_np [str] product name
  inventory [dict] inventory dictionary
  """
  #verify quantity availability
  is_valid_quantity=False
  while not is_valid_quantity:

    try:
      sell_quantity=_is_valid_input("Quantità: ","int")
      assert(sell_quantity <= inventory["prodotto"][sell_np]["quantità"])
      return sell_quantity

    except AssertionError:
      #selected quantity not available
      #choose new quantity or stop transaction
      print("La quantità disponibile è inferiore a quella venduta")
      print(f"Quantità di {sell_np} disponibile: {inventory['prodotto'][sell_np]['quantità']}")
      retry_input_quantity=input("Proseguire con una quantità diversa?: [si/No] ")

      if retry_input_quantity.lower()=="no":
        return 0

**5. User Command Function**

This section includes all function related to allowed user command.


The main function is named "actions()" that invokes different private functions based on user input

In [ ]:
#Dictionarie of all possible user command
#keys are user possible command
#values is the description of command

command_dict={"aiuto":"mostra i possibili comandi",
              "aggiungi":"aggiungi un prodotto al magazzino",
              "elenca":"elenca i prodotti in magazzino",
              "vendita":"registra una vendita effettuata",
              "profitti":"mostra i profitti totali",
              "chiudi":"esci dal programma"}


def _aiuto():
  """
  Shows all possible command
  """
  print("I comandi disponibili sono i seguenti:")
  for com in command_dict.keys():
    print(f"-{com}: {command_dict[com]}")


def _close():
  """
  Close software
  """
  print("Bye Bye")


def _show_profit():
  """
  Show total profit
  """
  cash_flow=_load_file("cash_register.json")
  print(f"Profitto: lordo=€{round(cash_flow['Profitto']['lordo'],2)} netto=€{round(cash_flow['Profitto']['netto'],2)}")


def _add_product():
  """
  Add a new product to inventory
  """
  #collect new product name and quantity
  np_name=input("Nome del prodotto: ") #np=new product
  np_quantity=_is_valid_input("Quantità: ","int")

  #load inventory data
  inventory=_load_file("inventory.json")
  #verify if the new product has been alredy registered
  if np_name in inventory["prodotto"].keys():
    inventory["prodotto"][np_name]["quantità"]+=np_quantity

  #first encounter of new product, required purchace and sell price
  else:
    np_purch_price="€"+str(_is_valid_input("Prezzo di acquisto: ","float"))
    np_sell_price="€"+str(_is_valid_input("Prezzo di vendita: ","float"))
    inventory["prodotto"][np_name]={"quantità":np_quantity,"prezzo di acquisto":np_purch_price,"prezzo di vendita":np_sell_price}

  #saving results
  _update_file(inventory,"inventory.json")
  print(f"Aggiunto {np_quantity} X {np_name}")


def _show_product():
  """
  Show all products registered in inventory
  """
  #load inventory file
  inventory=_load_file("inventory.json")

  if inventory["prodotto"]!={}: #verify that inventory is not empty
    max_len_product, max_len_quantity, max_len_price=_find_max_string_len(inventory)
    print(_adjust_string("Prodotto",max_len_product),_adjust_string("Quantità",max_len_quantity),_adjust_string("Prezzo",max_len_price))
    for item in inventory["prodotto"].keys():
      print(_adjust_string(item,max_len_product),
            _adjust_string(str(inventory["prodotto"][item]["quantità"]),max_len_quantity),
            _adjust_string(str(inventory["prodotto"][item]["prezzo di vendita"]),max_len_price))
  else:
    print("Inventario vuoto!")


def _store_transaction(sell_np,sell_quantity,i_trans,inventory,transaction_dict):
  """
  Update trasaction_dict with transaction number i_trans

  sell_np [str] product name
  sell_quantity [int] product quantity
  i_trans [int] transaction number
  inventory [dict] inventory dictionary
  transaction_dict [dict] transaction dictionary
  """
  #retrive purchase/sell price from inventory
  purch_price=float((inventory["prodotto"][sell_np]["prezzo di acquisto"]).replace("€",""))
  sell_price=float((inventory["prodotto"][sell_np]["prezzo di vendita"]).replace("€",""))

  #store transaction information
  transaction_dict[i_trans]={"prodotto":sell_np,
                      "prezzo di vendita":sell_price,
                        "prezzo di acquisto":purch_price,
                        "quantità":sell_quantity,
                        "profitto netto":(sell_quantity*(sell_price-purch_price)),
                        "profitto lordo":(sell_quantity*sell_price)}

  return transaction_dict


def _remove_sold_product(sold_product,i_trans):
  """
  Remove products from inventory after sale

  sold_product [transaction dictionary]
  i_trans [int] transaction number
  """
  #load inventory file
  inventory=_load_file("inventory.json")

  #Decrement sold product quantity
  qnt_i=sold_product[i_trans]["quantità"]
  prod_i=sold_product[i_trans]["prodotto"]
  inventory["prodotto"][prod_i]["quantità"]-=qnt_i

  #check if the product sold is still available after sale (minimum quantity 1)
  #if product sold remaining quantity = 0 remove product from inventory
  if inventory["prodotto"][prod_i]["quantità"]==0:
    del inventory["prodotto"][prod_i]

  #update inventory file
  _update_file(inventory,"inventory.json")



def _sell_product():

  """
  Sell a product
  """

  #initialize a dictionarie to store all transaction
  transaction_dict={}

  #while loop will continue register new transaction until user choose to stop it
  stop_transaction_bool=False
  #counter for transaction
  i_trans=0

  while not stop_transaction_bool:


    #load inventory
    inventory=_load_file("inventory.json")
    try:
      #Acquire sold product name
      sell_np=input("Nome del prodotto: ")
      #verify product exists
      assert(sell_np in inventory["prodotto"].keys())

      #Product is valid, ask for available quantity
      sell_quantity=_input_available_qauntity(sell_np,inventory)

      if sell_quantity!=0:
        #update counter
        i_trans+=1
        #store transaction
        transaction_dict=_store_transaction(sell_np,sell_quantity,i_trans,inventory,transaction_dict)

        #Decrement sold product quantity from inventory
        _remove_sold_product(transaction_dict,i_trans)

    #Error: Product not available in inventory
    except AssertionError:
      print("Il prodotto non è disponibile nell'inventario")
      pass

    #User input: continue selling or stop transaction
    stop_transaction=input("Aggiungere un altro prodotto? [si/No] ")
    if stop_transaction.lower()=="no":
      stop_transaction_bool=True

  #initialising variable for total and net earn
  total_earn=0
  net_earn=0

  #Transaction closed
  #show recap of transaction

  if transaction_dict!={}:
    print("Vendita registrata")
    for i_trans in transaction_dict.keys():
      qnt_i=transaction_dict[i_trans]["quantità"]
      prod_i=transaction_dict[i_trans]["prodotto"]
      profit_i=transaction_dict[i_trans]["profitto lordo"]

      #update profit
      total_earn+=profit_i
      net_earn+=transaction_dict[i_trans]["profitto netto"]

      #show all transactions and earns
      print(f"- {qnt_i} X {prod_i}: €{round(profit_i,2)}")
    #show total earn from transaction
    print(f"Totale: €{round(total_earn,2)}")

    #Load and update profit data
    cash_flow=_load_file("cash_register.json")
    cash_flow["Profitto"]["lordo"]+=total_earn
    cash_flow["Profitto"]["netto"]+=net_earn

    #store profit
    _update_file(cash_flow,"cash_register.json")

  else:
    print("Nessuna vendita effettuata")


def actions(user_command):
  """
  Function that accept in input a valid command

  Valid command:
  -aiuto: print all valid command
  -aggiungi: insert product necessary information:
    •Product name
    •Quantity
    •Purchase price
    •Sell price
  -elenca: show all product in inventory
  -vendita: register a new transaction
  -profitti: show total profit
  -chiudi: close the program

  user_command [str] user command
  """

  #Based on user command, the function invokes another function
  if user_command=="aiuto":
    _aiuto()
    print("\n")

  elif user_command=="elenca":
    _show_product()

  elif user_command=="aggiungi":
    _add_product()

  elif user_command=="vendita":
    _sell_product()

  elif user_command=="profitti":
    _show_profit()

  elif user_command=="chiudi":
    _close()

**Main**


Management software for vegan products shop

In [ ]:
#import list of possible user command
command_key=command_dict.keys()

#Input
#continue asking for a valid command
is_valid_command=False
while not is_valid_command:
  try:
    command=(input("Inserisci un comando: [aiuto] ")).lower()
    assert(command in command_key)
    if command=="aiuto":
      actions(command)
    else:
      is_valid_command=True

  except AssertionError:
    print("Comando non valido")
    actions("aiuto")


actions(command)